In [ ]:
#- - - - - - - - - - - - - - -  Import required modules/packages/dependencies- - - - - - - - - - - - - - - - - - - - - _
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import cartopy
import geopandas as gpd
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import os
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import folium
import ipywidgets
from IPython.display import HTML, display
import numpy as np

In [ ]:
#- - - - - - - - - - - - - - -  Initial Map SetUp- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -- - - -
 
plt.ion() # make the plotting interactive

# create a crs using ccrs.UTM() that corresponds to our CRS
myCRS = ccrs.UTM(29) # create a Universal Transverse Mercator reference system to transform our data.

myFig = plt.figure(figsize=(10, 10))  # create a figure of size 10x10 (representing the page size in inches)

ax = plt.axes(projection=myCRS)  # finally, create an axes object in the figure, using a UTM projection,
# where we can actually plot our data.


In [ ]:
outline = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/egm722_Practicals/egm722/week2/data_files/NI_outline.shp')) # load the outline of Northern Ireland for a backdrop


# first, we just add the outline of Northern Ireland using cartopy's ShapelyFeature
outline_feature = ShapelyFeature(outline['geometry'], myCRS, edgecolor='r', facecolor='w')
                     
xmin, ymin, xmax, ymax = outline.total_bounds
ax.add_feature(outline_feature)  # add the features we've created to the map.

# using the boundary of the shapefile features, zoom the map to our area of interest
ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=myCRS)  # because total_bounds
# gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.

ax.add_feature(cartopy.feature.OCEAN)
#ax.add_feature(cartopy.feature.BORDERS)
myFig



In [ ]:
#- - - - - - - - - - - - - - -  Add initial data- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -- - - -

# load the necessary data here 
counties = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/egm722_Practicals/egm722/week2/data_files/Counties.shp'))
wards = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/egm722_Practicals/egm722/week3/data_files/NI_Wards.shp'))
towns = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/egm722_Practicals/egm722/week2/data_files/Towns.shp'))
water = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/egm722_Practicals/egm722/week2/data_files/Water.shp'))
rivers = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/egm722_Practicals/egm722/week2/data_files/Rivers.shp'))
farmCensusDEA = gpd.read_file('c:/Carol_PG_CERT_GIS/data/FarmCensusDistrictElectoralArea2019.shp')
lakes= gpd.read_file('c:/Carol_PG_CERT_GIS/data/Lakes_of_Northern_Ireland.shp')



In [ ]:
print(lakes.columns)

In [ ]:
#a=farmCensusDEA.loc[(farmCensusDEA['TotalCrops']   < 100) & (farmCensusDEA['OBJECTID']     < 10), "TotalCrops"].sum()
lakes.head(-1)
lakes.loc[lakes["town"] == "THE DIAMOND"] #select based on 1: gdf.loc  [gdf['ColumnHeading'< 0] 
lakes.loc[(lakes["town"] == "THE DIAMOND") & (lakes['name'] == "Lough Neagh")] #select based on 2: gdf.loc[(gdf["colum"] == ("value") or gdf["colums"] == ("value"))]
lakes.loc[(lakes["town"] == "THE DIAMOND") & (lakes["county"] == "Tyrone") & (lakes['name'] == "Lough Neagh")]

In [ ]:
# pick colors, add features to the map
county_colors = ['firebrick', 'seagreen', 'royalblue', 'coral', 'violet', 'cornsilk']

# get a list of unique counties for the lake polys
lake_names = list(lakes.county.unique())
lake_names.sort()  # sort the counties alphabetically by name

# next, add the municipal outlines to the map using the colors that we've picked.
# here, we're iterating over the unique values in the 'CountyName' field.
# we're also setting the edge color to be black, with a line width of 0.5 pt.
# Feel free to experiment with different colors and line widths.
#for ii, name in enumerate(lakes):
    #feat = ShapelyFeature(lakes.loc[lakes['name'] == name, 'geometry'],  # first argument is the geometry
                          #myCRS,  # second argument is the CRS
                          #edgecolor='k',  # outline the feature in black
                          #facecolor=county_colors[ii],  # set the face color to the corresponding color from the list
                          #linewidth=1,  # set the outline width to be 1 pt
                          #alpha=0.25)  # set the alpha (transparency) to be 0.25 (out of 1)
    #ax.add_feature(feat)  # once we have created the feature, we have to add it to the map using ax.add_feature()

#ax.add_feature(farmCensusDEA)

In [ ]:
print(farmCensusDEA.shape) # 80 rows and 28 fields
print(farmCensusDEA.columns)
print(farmCensusDEA['TotalAreaF'].mean())
print(farmCensusDEA['TotalCrops'].sum())

In [ ]:
# generate matplotlib handles to create a legend of features in output map
def generate_handles(labels, colors, edge='k', alpha=1):
    lc = len(colors)  # get the length of the color list
    handles = []
    for i in range(len(labels)):
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[i % lc], edgecolor=edge, alpha=alpha))
    return handles

# here, we're setting the edge color to be the same as the face color. Feel free to change this around,
# and experiment with different line widths.
#water_feat = ShapelyFeature(water['geometry'],  # first argument is the geometry
                            #myCRS,  # second argument is the CRS
                            #edgecolor='mediumblue',  # set the edgecolor to be mediumblue
                            #facecolor='mediumblue',  # set the facecolor to be mediumblue
                            #linewidth=1)  # set the outline width to be 1 pt
#ax.add_feature(water_feat)  # add the collection of features to the map


Fcensus_feat = ShapelyFeature(farmCensusDEA['geometry'],  # first argument is the geometry
                            myCRS,  # second argument is the CRS
                            edgecolor='mediumblue',  # set the edgecolor to be mediumblue
                            facecolor='mediumblue',  # set the facecolor to be mediumblue
                            linewidth=1)  # set the outline width to be 1 pt




#lakes




river_feat = ShapelyFeature(rivers['geometry'],  # first argument is the geometry
                            myCRS,  # second argument is the CRS
                            edgecolor='royalblue',  # set the edgecolor to be royalblue
                            linewidth=0.2)  # set the linewidth to be 0.2 pt
ax.add_feature(river_feat)  # add the collection of features to the map

# ShapelyFeature creates a polygon, so for point data we can just use ax.plot()
town_handle = ax.plot(towns.geometry.x, towns.geometry.y, 's', color='0.5', ms=6, transform=myCRS)

# generate a list of handles for the county datasets
lake_handles = generate_handles(lakes.name.unique(), lake_colors, alpha=0.25)

# note: if you change the color you use to display lakes, you'll want to change it here, too
water_handle = generate_handles(['Water'], ['mediumblue'])

# note: if you change the color you use to display rivers, you'll want to change it here, too
river_handle = [mlines.Line2D([], [], color='royalblue')]  # have to make this a list

# update county_names to take it out of uppercase text
#nice_names = [name.title() for name in county_names]

# ax.legend() takes a list of handles and a list of labels corresponding to the objects you want to add to the legend
#for haneles- might want to add in county_handles in future: " = county_handles + water_handle + river_handle + town_handle "
handles = water_handle + river_handle + town_handle
#for labels- might want to add in nice names in fure -use "nice_names + ['Water', 'Rivers', 'Towns'] "
labels = ['Water', 'Rivers', 'Towns']

leg = ax.legend(handles, labels, title='Legend', title_fontsize=12,
                fontsize=10, loc='upper left', frameon=True, framealpha=1)

gridlines = ax.gridlines(draw_labels=True,  # draw  labels for the grid lines
                         xlocs=[-8, -7.5, -7, -6.5, -6, -5.5],  # add longitude lines at 0.5 deg intervals
                         ylocs=[54, 54.5, 55, 55.5])  # add latitude lines at 0.5 deg intervals
gridlines.left_labels = False  # turn off the left-side labels
gridlines.bottom_labels = False  # turn off the bottom labels

# add the text labels for the towns
for ind, row in towns.iterrows():  # towns.iterrows() returns the index and row
    x, y = row.geometry.x, row.geometry.y  # get the x,y location for each town
    ax.text(x, y, row['TOWN_NAME'].title(), fontsize=8, transform=myCRS)  # use plt.text to place a label at x,y

# add the scale bar to the axis
scale_bar(ax)

# save the figure as map_ipythonTest.png, cropped to the axis (bbox_inches='tight'), and a dpi of 300
myFig.savefig('map_ipythoTest.png', bbox_inches='tight', dpi=300)

myFig



In [ ]:


#- - - - - - - - - - - - - - -  Carry out spatial analyses- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#oneKmBuffer = bike_station_locations.geometry.buffer(1)
#oneKmBuffer.head()
#print(oneKmBuffer.head())
#for index, row in bike_station_locations.iterrows():



#TODO: define and code for further analyses



# your analysis goes here...counties have been removed

join = gpd.sjoin(counties, wards, how='inner', lsuffix='left', rsuffix='right') # perform the spatial join of counties and wards
joinedData = join.groupby(['CountyName'])['Population'].sum() # assign summary data to variable to be printed
print(joinedData)
# below here, you may need to modify the script somewhat to create your map.






# to make a nice colorbar that stays in line with our map, use these lines:
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1, axes_class=plt.Axes)


# plot the ward data into our axis, using
ward_plot = wards.plot(column='Population', ax=ax, vmin=1000, vmax=8000, cmap='viridis',
                       legend=True, cax=cax, legend_kwds={'label': 'Resident Population'})


county_outlines = ShapelyFeature(counties['geometry'], myCRS, edgecolor='r', facecolor='none')


ax.add_feature(county_outlines)
county_handles = [mpatches.Rectangle((0, 0), 1, 1, facecolor='none', edgecolor='r')]

ax.legend(county_handles, ['County Boundaries'], fontsize=12, loc='upper left', framealpha=1)

# save the figure
fig.savefig('sample_map22.png', dpi=300, bbox_inches='tight')

fig


#- - - - - - - - - - - - - - -  Present results from analysis- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#TODO: present results


#- - - - - - - - - - - - - - - Output the final map- - - - - - - - - - - - - - -

# add gridlines below
gridlines = ax.gridlines(draw_labels=True,
                         xlocs=[-8, -7.5, -7, -6.5, -6, -5.5],
                         ylocs=[54, 54.5, 55, 55.5],
                         linewidth = 0.5, color = 'gray', alpha = 0.5, linestyle='--')

gridlines.right_labels = False
gridlines.bottom_labels = False
gridlines.left_labels = True
gridlines.top_labels = True

In [ ]:
#-----------------------------How to add generic layers from cartopy:--------------------------------------------------------------------------
#ax.add_feature(cartopy.feature.OCEAN)
#ax.add_feature(cartopy.feature.BORDERS)
#ax.add_feature(cartopy.feature.RIVERS)

In [ ]:
#-----------------------------How to load in data:--------------------------------------------------------------------------
#roads = gpd.read_file('data_files/NI_roads.shp')
#ax.add_feature(cartopy.feature.OCEAN)

In [ ]:
#-----------------------------How to add and display points and get CRS:--------------------------------------------------------
#pt = Point(-6.677, 55.150) # creates a 2d point with coordinates -6.677, 55.150
#pt2 = Point(-6.658, 55.213) # creates a 2d point with coordinates -6.658, 55.213
#use any of the following to get point co-ords:
#print(pt.svg)
#print(pt.coords[:])
#print(list(pt.coords))
#print(pt)


#points.crs

In [ ]:
#-----------------------------How to add and display lines & line methods:--------------------------------------------------------
#line1 = LineString([pt, pt2]) # method one of creating a LineString, using a list of Point objects
#line2 = LineString([(-6.677, 55.150), (-6.658, 55.213)]) # method two, using a list of coordinate tuples

#print(line1)
#print(line2)

#line1.equals(line2) # check to see if these are the same geometry

#print(line1.length)
#print(line1.centroid)

#print(line1.project(center) / line1.length) # check to see how far along the line our centerpoint is

#print(center) # print the WKT representation of the center point
#print(line1.interpolate(0.5, normalized=True)) # print the WKT representation of the point 50% along the line


In [ ]:
#-----------------------------How to add and display polys methods:-----------------------------------
#poly1 = Polygon([(-6.677, 55.150), (-6.658, 55.213), (-6.722, 55.189)])
#poly2 = Polygon([pt, pt2, Point(-6.722, 55.189)])

#print(poly1) # print a wkt representation of the polygon
#print(poly2)
#print(poly1.equals(poly2))
----
#polygon_with_hole = Polygon(shell=[(-6.677, 55.150), (-6.658, 55.213), (-6.722, 55.189)],
                            holes=[[(-6.684, 55.168), (-6.704, 55.187), (-6.672, 55.196)]]) # note the double brackets

#print(polygon_with_hole)

#print(polygon_with_hole.exterior) # this is a single LinearRing
#for lr in polygon_with_hole.interiors: # this is potentially multiple LinearRing objects
#    print(lr)

In [ ]:
#-----------------------------How to get poly area/length bound geometry:-----------------------------------
#print('perimeter: ', poly1.length) # print the perimeter
#print('area: ', poly1.area) # print the area
#print('centroid: ', poly1.centroid) # get the centerpoint of the rectangle
#print('bounding coordinates: ', poly1.bounds) # get the minimum x, minimum y, maximum x, maximum y coordinates
#print('bounding box: ', poly1.envelope) # get the minimum bounding rectangle of the polygon, parallel to the coordinate axes
#print('rotated bounding box: ', poly1.minimum_rotated_rectangle) # get the smallest possible rectangle that covers the polygon

In [ ]:
#-----------------------------How to display points with a single symbol:--------------------------------------------------------
# We don't seem to need to use ShapelyFeature as that looks to be for lines and polys
# So for Point data, we can just use ax.plot() directly.  The code below will add a gray (color='0.5') 
# square ('s') marker of size 6 (ms=6) at each x, y location:

#ax.plot(gdf.geometry.x, gdf.geometry.y, 's', color='0.25', ms=6, transform=myCRS)
#ax.plot(cities.geometry.x, cities.geometry.y, 'o', color='r', ms=6, transform=myCRS)

#You can create handles a the same time:
#town_handle = ax.plot(towns_only.geometry.x, towns_only.geometry.y, 's', color='0.25', ms=6, transform=myCRS)
#city_handle = ax.plot(cities.geometry.x, cities.geometry.y, 'o', color='r', ms=6, transform=myCRS)
#point symbols: s= square, o= circle
#point colours: 'r'= red, 0.25' = dark grey
#more info on markers and colours here:https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.plot.html

In [ ]:
#-----------------------------How to display lines with a single symbol:--------------------------------------------------------
#gdf = ShapelyFeature(gdf['geometry'], # first argument is the geometry
                            #myCRS, # second argument is the CRS
                            #edgecolor='dodgerblue', # set the edgecolor to be royalblue
                            #linewidth=0.2) # set the linewidth to be 0.2 pt
#ax.add_feature(gdf) # add the collection of features to the map

#myFig # to show the updated figure

In [ ]:
#-----------------------------How to display polys with a single symbol:--------------------------------------------------------
# here, we're setting the edge color to be the same as the face color. Feel free to change this around,
#gdf = ShapelyFeature(gdf['geometry'], # first argument is the geometry
                            #myCRS, # second argument is the CRS
                            #edgecolor='lightblue', # set the edgecolor to be mediumblue
                            #facecolor='lightblue', # set the facecolor to be mediumblue
                            #linewidth=1) # set the outline width to be 1 pt
#ax.add_feature(gdf) # add the collection of features to the map

    
#myFig # to show the updated figure

In [ ]:
#-----------------------------How to display polys by colours based upon field values:--------------------------------------------------------------------------
#see week2 ipynb
# get the number of unique municipalities we have in the dataset
#num_counties = len(counties.CountyName.unique())
#list_counties = counties.CountyName.unique()
#county_names = list(counties.CountyName.unique())
#print('Number of unique features: {}'.format(num_counties)) # note how we're using {} and format here!
#print('Names of counties: {}'.format(list_counties))# pick colors for the individual county boundaries - make sure to add enough for each of the counties
# to add a color, enclose the name above (e.g., violet) with single (or double) quotes: 'violet'
# remember that each colors should be separated by a comma
#county_colors = ['lightsteelblue', 'lightcyan', 'lightyellow', 'lightpink','lavenderblush', 'papayawhip']

# get a list of unique names for the county boundaries
#county_names = list(counties.CountyName.unique())
#county_names.sort() # sort the counties alphabetically by name

# next, add the municipal outlines to the map using the colors that we've picked.
# here, we're iterating over the unique values in the 'CountyName' field.
# we're also setting the edge color to be black, with a line width of 0.5 pt. 
# Feel free to experiment with different colors and line widths.
#for ii, name in enumerate(county_names):
    #feat = ShapelyFeature(counties.loc[counties['CountyName'] == name, 'geometry'], # first argument is the geometry
                          #myCRS, # second argument is the CRS
                          #edgecolor='k', # outline the feature in black
                          #facecolor=county_colors[ii], # set the face color to the corresponding color from the list
                          #linewidth=1, # set the outline width to be 1 pt
                          #alpha=0.25) # set the alpha (transparency) to be 0.25 (out of 1)
    #ax.add_feature(feat) # once we have created the feature, we have to add it to the map using ax.add_feature()

#myFig # to show the updated figure

In [ ]:
#-----------------------------How to add text lables to points:-------------------------------------------------------------------

#for ind, row in gdf.iterrows(): # gdf.iterrows() returns the index and row
    #x, y = row.geometry.x, row.geometry.y # get the x,y location for each town
    #ax.text(x, y, row['LabelColumn'].title(), fontsize=7, transform=myCRS) # use 
    #plt.text #to place a label at x,y


In [ ]:
#-----------------------------How to add column to a geodataframe(points/lines/polys):------------------------------
#see prac3 ipynb:
#for ind, row in gdf.iterrows(): # iterate over each row in the GeoDataFrame
    #gdf.loc[ind, 'Length'] = row['geometry'].length # assign the row's geometry length to a new column, Length
    
#print(gdf.tail()) # print the updated GeoDataFrame to see the changes

In [ ]:
#-----------------------------How to rename columns-----------------------------
gdf.rename(columns={"OldName":"NewName"},inplace=True)
print(dgf.columns)


In [ ]:
#-----------------------------How to get unique values for a data field:--------------------------------------------------------------------------num_counties = len(counties.CountyName.unique())
#newvar = gdf.columheading.unique()# create a new var containing unique values from within a field
#newlist = list(gdf.columheading.unique())# create a new list containing unique values from within a field
#newlist.sort() # sort the unique field values alphabetically 

In [ ]:
#-------------------How to Return whole row for a fetaure with max value:------
#df[df['Value']==df['Value'].max()]

In [ ]:
#-----------------------------How to select features:--------------------------------------------------------------------------
    
#gdf.head()
#gdf.loc[0:2] #shows the first 3 rows
#gdf.loc[farmCensusDEA['ColumnName'] < 100]#shows total crops <100 gdf.loc[gdf['ColumnHeading'] 
#to select records based upon 2 variable: 
#gdf.loc[(gdf['columheading1']> 435) & (dgf['columheading2'] < 1)]
#gdf.loc[(gdf['ColumnName']   < 100) & (gdf['ColumnName']     < 10)]
          
#to summarise a subset of records based upon 2 variable: 
#gdf.loc[(gdf['columheading1']> 435) & (gdf['columheading1'] < 1), 'ColumnToSummarise'].sum()
#gdf.loc[(gdf['ColumnName']   < 100) & (gdf'ColumnName']     < 10), 'ColumnName'].sum()
#a=gdf.loc[(gdf['ColumnName']   < 100) & (gdf['ColumnName']     < 10), 'ColumnName'].sum()

#print("Total x in areas that meet conditions ={}".format(a.astype(int)))


In [ ]:
#---------How to summarise features/selected feature attributes:----------------

#new var = df['columheading'].sum example below- this sums all records
#sum_roads = roads_itm['Length'].sum()

#below- this sums a selection of records
#new var = df[df['columheading'] == 'Value']['Fieldto Summarise'].sum example 
#sum_motorway = roads_itm[roads_itm['Road_class'] == 'MOTORWAY']['Length'].sum()
#print('{:.2f} total m of roads'.format(sum_roads))
#print('{:.2f} total m of motorway'.format(sum_motorway))


In [ ]:
#-----How to summarise features/selected feature attributes using GROUPBY:-----
#GroupBy one field:
#gdf.groupby(['FieldtoGroupby'])['FieldToSumariseBy'].sum() - example below
#roads_itm.groupby(['Road_class'])['Length'].sum() / 1000 # convert to km

#GroupBy two fields:
#gdf.groupby(['GroupByField1', 'GroupByField2'])['Length'].sum() / 1000- example below:
#join.groupby(['CountyName', 'Road_class'])['Length'].sum() / 1000


#GroupBY one field using selected records only:

#gdf[gdf['SelectionField'] == 'Value'].groupby(['GroupByField'])['SummaryField'].sum() 
# example below:
#roads_itm[roads_itm['Road_class'] == 'MOTORWAY'].groupby(['Road_class'])['Length'].sum() / 1000 # convert to km

#GroupBY two fields using selected records only:
#gdf[gdf['SelectionField'] == 'Value'].groupby(['Field1', 'Field2'])['FieldToSummarise'].sum() 

In [ ]:
#-----------------------------How to.....Spatial Analyses--------------------------
# Always ensure the CRS of the gdf layers are the same! to check:
# print(gdf1.crs == dgf2.crs) #this should return 'True'

#BUFFER:
#pt_buffer = pt.buffer(0.001) # buffer the point by 0.001 in the same coordinates

#CONTAINMENT:
#poly.contains(pt1)# will give true or false

#INTERSECT:
#print(line1.intersects(line2)) # intersects() returns True if the geometries touch/intersect/overlap, False otherwise
#print(line1.intersects(poly))
#print(line1.intersection(line2)) # if the geometries intersect, this will be the Point(s) of intersection

#SPATIAL JOIN:
#join = gpd.sjoin(gdf1, gdf2, how='inner', lsuffix='left', rsuffix='right') # perform the spatial join
#join # shows the joined table- join type is a gdf- you can check this with type()


#CLIP:
#Simple clip:
#newClipped_gdf = gpd.clip(gdf1, gdf2)# clip the gdf1 by gdf2
#Clip by selected feature(s):
#newClipped_gdf = gpd.clip(gdf1, gdf2[gdf2['FieldToSelectfeatures'] == value]) # clip dgf1 by selected feature(s) in gdf2 
 
#see prac3.ipynb for how to create a new clipped layer by itterating through 
# selected records with different values, eg- clipping by count and then concatenating
#results into a new layer

#UNION:
#newgdf = gdf['geometry'].unary_union #creates a single polygon (or multipolygon)
#composed of the individual polygons.  For example:
#outline = counties['geometry'].unary_union
#more details here:https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.unary_union.html

#INTERSECTION:
#intersection_gdf = gdf.intersection(row['geometry']) #Refer to `shapely.intersection` for full documentation.

In [ ]:
#-----------------------------How to join fields:------------------------

#join a csv with a shapefile using a joinfield
new_gdf =gpd.GeoDataFrame(exisiting_csv.merge(gdf, on="joinField"))
new_gdf.head()

In [ ]:
#-----------------------------How to change field type:------------------------

# Set the columns data type to integer instead of string
#gdf['columnName'] = gdf['columnName'].astype(int)
# can use .astype(str), .astype(float),.astype(list) etc.

In [ ]:
#-----------------------------How to create LU tables:------------------------
# one way os to produce a dictionary object of key/value pairs:
# dict = {1:a,
#         2:b,
#         3:c}
#
#you can define a function to take an array and return a dictionary object- 
#containing the count for wach value in the array:
#def count_unique(array, names, nodata=0):
#    '''
#    Count the unique elements of an array.

#    :param array: Input array
#    :param names: a dict of key/value pairs that map raster values to a name
#    :param nodata: nodata value to ignore in the counting
    
#    :returns count_dict: a dictionary of unique values and counts
#    '''
#    count_dict = dict() # create the output dict
#    for val in np.unique(array): # iterate over the unique values for the raster
#        if val == nodata: # if the value is equal to our nodata value, move on to the next one
#            continue
#        count_dict[names[val]] = np.count_nonzero(array == val)
#    return count_dict # return the now-populated output dict
#
#newDict = count_unique(gdf, fieldname)
#print(newDict) # show the results
#######
#
#
#or try:
#county_dict = dict()
#for ind, row in counties.iterrows():
#    county_dict[row['CountyName'].title()] = county_stats[ind] # we're using str.title() because we're not shouting.
#
#print(county_dict['Down']) # this should be the same as the output for the previous cell
#
#
#or see Prac5 ipynb (section3) to use the built in zip function & https://docs.python.org/3.8/library/functions.html#zip

#Working with dictionary - see pract5 ipynb

In [ ]:
#-----------------------------How to blur outside of NI:------------------------
#First create a bounding box layer, 
#map_frame = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)]) # create polygon of map extent
#then union the counties a
#counties_union = unary_union(counties.geometry) # create NI outline by joining geometries of counties
#and then create a map overlay poly with counties 'hole' removed and set layer transparency

#mapblur = ShapelyFeature(map_frame.symmetric_difference(counties_union), myCRS, facecolor='w', alpha=0.5)

#ax.add_feature(mapblur)

In [ ]:
#-----------------------------How to create a legend and labels:----------------------------------------------------------------
#see week2 ipynb
# generate a list of handles for the county datasets
# first, we add the list of names, then the list of colors, and finally we set the transparency
# (since we set it in the map)
#county_handles = generate_handles(counties.CountyName.unique(), county_colors, alpha=0.25)

# note: if you change the color you use to display lakes, you'll want to change it here, too
#water_handle = generate_handles(['Lakes'], ['dodgerblue'])

# note: if you change the color you use to display rivers, you'll want to change it here, too
#river_handle = [mlines.Line2D([], [], color='dodgerblue')]
# update county_names to take it out of uppercase text
#nice_names = [name.title() for name in county_names]
#print(counties.CountyName.unique())

#print(nice_names)
# ax.legend() takes a list of handles and a list of labels corresponding to the objects 
# you want to add to the legend
#handles = county_handles + water_handle + river_handle + town_handle + city_handle # use '+' to concatenate (combine) lists
#labels = nice_names + ['Lakes', 'Rivers', 'Towns', 'Cities']

#leg = ax.legend(handles, labels, title='Legend', title_fontsize=14, 
                 #fontsize=10, loc= 'upper left', frameon=True, framealpha=1)

#myFig # to show the updated figure

In [ ]:
#-----------------------------How to create gridlines:--------------------------------------------------------------------------
#gridlines = ax.gridlines(draw_labels=True, # draw  labels for the grid lines
                         #xlocs=[-7.5, -7, -6.5, -6], # add longitude lines at 0.5 deg intervals
                         #ylocs=[54.5, 55, 55.5]) # add latitude lines at 0.5 deg intervals
#gridlines.left_labels = False # turn off the left-side labels
#gridlines.bottom_labels = False # turn off the bottom labels

#myFig # to show the updated figure

In [ ]:
#-----------------------------How to add a scale bar :--------------------------------------------------------------------------
#def scale_bar(ax, location=(0.92, 0.95)):
    #x0, x1, y0, y1 = ax.get_extent()
    #sbx = x0 + (x1 - x0) * location[0]
    #sby = y0 + (y1 - y0) * location[1]

    #ax.plot([sbx, sbx - 10000], [sby, sby], color='k', linewidth=7, transform=ax.projection)
    #ax.plot([sbx, sbx - 5000], [sby, sby], color='k', linewidth=6, transform=ax.projection)
    #ax.plot([sbx-5000, sbx - 10000], [sby, sby], color='w', linewidth=6, transform=ax.projection)
    #ax.plot([sbx-9000, sbx - 10000], [sby, sby], color='k', linewidth=6, transform=ax.projection)

    #ax.text(sbx, sby-4500, '10 km', transform=ax.projection, fontsize=6)
    #ax.text(sbx-4250, sby-4500, '5', transform=ax.projection, fontsize=6)#should be 6250 but doesn't place well
    #ax.text(sbx-9750, sby-4500, '1', transform=ax.projection, fontsize=6)# should be 11250 but doesn't place well
    #ax.text(sbx-12500, sby-4500, '0', transform=ax.projection, fontsize=6)
    ##ax.text(sbx-24500, sby-4500, '0 km', transform=ax.projection, fontsize=6)

    
#scale_bar(ax)#scale_bar(ax)

In [ ]:
#-----------------------------Working with CRS:--------------------------------
#to Convert dgf to ITM:
#gdf = gdf.to_crs(epsg=2157)

#if the dataset has a Geographic coordinate reference system (eg not ITM, 
# such as EPSG:4326- we will need to convert to a projected CRS, so that 
# attributes like length make sense! 

#to transform the GeoDataFrame to a different CRS, we have to provide either 
#a CRS or an EPSG code. We can also choose to do this in place (inplace=True), 
#or assign the output to a new GeoDataFrame object (inplace=False). Let's 
#transform the GeoDataFrame to Irish Transverse Mercator, and assign the output
#to a new variable, roads_itm.

#Rather than trying to find the correct JSON or PROJ representation of this CRS, we can instead use the EPSG code, which can be easier to work with.

#roads_itm = roads.to_crs(epsg=2157) # replace XX with the correct EPSG code for Irish Transverse Mercator

#print(roads.head()) to see the values for geometry based on old CRS
#print(roads_itm.head())to see the values for geometry based on new CRS
#now when we access length/ area attributes they will be in meters


#EPSG codes (originally organized by the European Petroleum Survey Group) 
#are a common way of working with coordinate reference systems. Each CRS in 
#the EPSG registry has a unique code and standard well-known text 
#representation.
#The crs attribute of the GeoDataFrame is actually a pyproj.CRS object. pyproj
#is a python interface to the PROJ library, which is a software for transforming
#geospatial coordinates from one CRS to another.

#Each pyproj.CRS object provides a number of methods for converting to different
#formats, including well-known text, EPSG codes, JavaScript Object Notation 
#(JSON), and PROJ string (i.e., '+proj=longlat +datum=WGS84 +no_defs +type=crs').

#For example, to see the JSON representation of the CRS, we would use the 
#pyproj.CRS.to_json() method:

#roads.crs.to_json() # show the representation of the CRS in JSON format

#more details in Practical3.ipynb

In [ ]:
#-----------------------------How to save map:--------------------------------------------------------------------------
#myFig.savefig('map5.png', bbox_inches='tight', dpi=300)

In [ ]:
#-----------------------------Miscellaneous:----------------------------------

#How to get min area covered by a gdf:
#minAreaVar = gdf.minimum_rotated_rectangle



In [ ]:


''''# - - - - - - - - - - - - - - -  Create the Folium map- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# this creates the map and centres the map on the mean x,y of all bike station locations
#map = folium.Map(location=[bike_station_locations.Latitude.mean(), bike_station_locations.Longitude.mean()],
                 zoom_start=14, control_scale=True)

# - - - - - - - - - - - - - - -  Add data to the map- - - - - - - - - - - - - -- - - - - - - - - - - - - - - - - - - - -
# this adds the bike station point locations to the map and uses the location field as a popup label
#for index, row in bike_station_locations.iterrows():
#    lat = row['Latitude']
#    long = row['Longitude']
#    name = row['Location']

 #   folium.Marker([lat, long], name).add_to(map)

# add data layers control
#folium.LayerControl().add_to(map)
# TODO: configure widget
#- - - - - - - - - - - - - - - Finishing Touches- - - - - - - - - - - - - - -- - - - - - - - - - - - - - - - - - - - - -
#this adds a title to the map   
#mapTitle = 'Belfast Bike Locations'
#title_html = '''
#             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
#            '''.format(mapTitle)
#
#map.get_root().html.add_child(folium.Element(title_html))

#- - - - - - - - - - - - - - - Save the Folium map- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
##this saves the map as html
#map.save("Belfast bikes.html")